In [1]:
# Data analytical multi-agent workflow using OpenAI API

import os
import re
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

In [2]:
# Load environment variables
load_dotenv()

# Check if the API key is set
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("API key is not set")

# Initialize the OpenAI client
client = OpenAI()

In [3]:
# Define the agents' prompts
PLANNING_AGENT_PROMPT = """You are a data analytical planning assistant. Given a dataset and its description,
your task is to provide specific and simple analysis plans, detailed instructions,
and suggested Python code that can later be given to a separate Python agent to generate
the Python code for executing the analysis plan.
Do not create figures.

Return output with the following format:

## Total number of steps:

## Step 1:
"""

In [4]:
PYTHON_AGENT_PROMPT = """You are a Python coding assistant that only outputs Python code without any explanations or comments.
Given an instruction and the suggested Python code, return the correct Python code."""

In [6]:
SUMMARIZATION_AGENT_PROMPT = """You are an analysis summarization assistant.
Given a dataset's description and the analysis results. Provide an analysis report."""

In [8]:
def run_analysis_planning_agent(query):
    """
    Sends a user query to the planning agent and returns the response.
    """
    print("### Run Planning agent")
    print(f"User query: {query}")
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": PLANNING_AGENT_PROMPT},
                {"role": "user", "content": query}
            ]
        )
        result = response.choices[0].message.content
        return result
    except Exception as e:
        print(f"Request failed: {e}. Please check your request.")
        return None

In [9]:
class PythonAgentWorkflow:
    def __init__(self):
        pass

    def extract_pattern(self, text, pattern):
        """
        Extracts a pattern from the given text.
        """
        match = re.search(pattern, text, flags=re.DOTALL)
        if match:
            return match.group(1).strip()
        return None

    def extract_step_i(self, planning_result, i, n_step):
        """
        Extracts the content of a specific step from the planning result.
        """
        if i < n_step:
            pattern = rf'## Step {i}:(.*?)## Step {i+1}'
        elif i == n_step:
            pattern = rf'## Step {i}:(.*)'
        else:
            print(f"Invalid step number {i}. It should be between 1 and {n_step}.")
            return None

        step_i = self.extract_pattern(planning_result, pattern)
        if not step_i:
            print(f"Failed to extract Step {i} content.")
            return None

        return step_i

    def extract_code(self, python_agent_result):
        """
        Extracts Python code from the response content.
        """
        retry = False
        print("### Extracting Python code")
        python_code = self.extract_pattern(python_agent_result, r'```python(.*?)```')
        if not python_code:
            retry = True
            print("Python function failed to generate or wrong output format. Setting retry to True.")

        return python_code, retry

    def run_python_agent(self, query):
        """
        Sends a user query to the Python agent and returns the response.
        """
        print("### Run Python agent")
        print(f"User query: {query}")
        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": PYTHON_AGENT_PROMPT},
                    {"role": "user", "content": query}
                ]
            )
            result = response.choices[0].message.content
            return result
        except Exception as e:
            print(f"Request failed: {e}. Please check your request.")
            return None

    def check_code(self, python_function, state):
        """
        Executes the Python function and checks for any errors.
        """
        retry = False
        try:
            print(f"### Python function to run: {python_function}")
            exec(python_function, state)
            print("Code executed successfully.")
        except Exception:
            print(f"Code failed.")
            retry = True
            print("Setting retry to True")
        return retry

    def process_step(self, planning_result, i, n_step, max_retries, state):
        """
        Processes a single step, including retries.
        """
        retry = True
        j = 0
        while j < max_retries and retry:
            print(f"TRY # {j}")
            j += 1
            step_i = self.extract_step_i(planning_result, i, n_step)
            if step_i:
                print(step_i)
                python_agent_result = self.run_python_agent(step_i)
                python_code, retry = self.extract_code(python_agent_result)
                print(python_code)
                retry = self.check_code(python_code, state)
        return None

    def workflow(self, planning_result):
        """
        Executes the workflow for processing planning results.
        """
        state = {}
        print("### ENTER WORKFLOW")
        n_step = int(self.extract_pattern(planning_result, '## Total number of steps:\\s*(\\d+)'))
        for i in range(1, n_step + 1):
            print(f"STEP # {i}")
            self.process_step(planning_result, i, n_step, max_retries=2, state=state)

        print("### Exit WORKFLOW")
        return None


In [10]:
# Load the 2017 World Happiness Report CSV file
file_path = "WHR_2017.csv"  # Make sure this file is in the same directory as your notebook
data = pd.read_csv(file_path)

In [11]:
# Display the first few rows and basic information about the dataset
print(data.head())
print(data.info())

       Country  Happiness.Rank  Happiness.Score  Whisker.high  Whisker.low  \
0       Norway               1            7.537      7.594445     7.479556   
1      Denmark               2            7.522      7.581728     7.462272   
2      Iceland               3            7.504      7.622030     7.385970   
3  Switzerland               4            7.494      7.561772     7.426227   
4      Finland               5            7.469      7.527542     7.410458   

   Economy..GDP.per.Capita.    Family  Health..Life.Expectancy.   Freedom  \
0                  1.616463  1.533524                  0.796667  0.635423   
1                  1.482383  1.551122                  0.792566  0.626007   
2                  1.480633  1.610574                  0.833552  0.627163   
3                  1.564980  1.516912                  0.858131  0.620071   
4                  1.443572  1.540247                  0.809158  0.617951   

   Generosity  Trust..Government.Corruption.  Dystopia.Residual  
0 

In [12]:
# Create a description of the dataset for the planning agent
dataset_description = """
The dataset is the World Happiness Report for 2017, which ranks 155 countries by their happiness levels.
It contains the following columns:
- Country: The name of the country
- Happiness Rank: The country's ranking based on the Happiness Score
- Happiness Score: A metric measured by asking sample people to rate their happiness on a scale of 0 to 10
- Whisker.high: The whisker high score for this country
- Whisker.low: The whisker low score for this country
- Economy (GDP per Capita): The extent to which GDP contributes to the calculation of the Happiness Score
- Family: The extent to which family contributes to the calculation of the Happiness Score
- Health (Life Expectancy): The extent to which life expectancy contributes to the calculation of the Happiness Score
- Freedom: The extent to which freedom contributes to the calculation of the Happiness Score
- Trust (Government Corruption): The extent to which perception of government corruption contributes to calculation of the Happiness Score
- Generosity: The extent to which generosity contributes to the calculation of the Happiness Score
- Dystopia Residual: The extent to which dystopia residual contributes to the calculation of the Happiness Score

Please provide a comprehensive analysis plan for this dataset, including basic statistical analysis,
exploration of relationships between variables, and identification of key factors influencing happiness scores.
"""

In [13]:
# Run the planning agent with the new dataset description
planning_result = run_analysis_planning_agent(dataset_description)
print(planning_result)

### Run Planning agent
User query: 
The dataset is the World Happiness Report for 2017, which ranks 155 countries by their happiness levels.
It contains the following columns:
- Country: The name of the country
- Happiness Rank: The country's ranking based on the Happiness Score
- Happiness Score: A metric measured by asking sample people to rate their happiness on a scale of 0 to 10
- Whisker.high: The whisker high score for this country
- Whisker.low: The whisker low score for this country
- Economy (GDP per Capita): The extent to which GDP contributes to the calculation of the Happiness Score
- Family: The extent to which family contributes to the calculation of the Happiness Score
- Health (Life Expectancy): The extent to which life expectancy contributes to the calculation of the Happiness Score
- Freedom: The extent to which freedom contributes to the calculation of the Happiness Score
- Trust (Government Corruption): The extent to which perception of government corruption contri

In [14]:
# Execute the workflow
import sys
import io

class Tee(io.StringIO):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.original_stdout = sys.stdout

    def write(self, data):
        self.original_stdout.write(data)
        super().write(data)

    def flush(self):
        self.original_stdout.flush()
        super().flush()

tee_stream = Tee()
sys.stdout = tee_stream

Python_agent = PythonAgentWorkflow()
Python_agent.workflow(planning_result)

sys.stdout = tee_stream.original_stdout
captured_output = tee_stream.getvalue()

# Summarization
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": SUMMARIZATION_AGENT_PROMPT},
        {"role": "user", "content": dataset_description + captured_output}
    ]
)
result = response.choices[0].message.content

print(result)

### ENTER WORKFLOW
STEP # 1
TRY # 0
### Task: Load the dataset and display the first few rows to understand its structure.
### Python Code:
```python
import pandas as pd

# Load the dataset
happiness_df = pd.read_csv('world_happiness_report_2017.csv')

# Display the first few rows of the dataset
print(happiness_df.head())
```
### Run Python agent
User query: ### Task: Load the dataset and display the first few rows to understand its structure.
### Python Code:
```python
import pandas as pd

# Load the dataset
happiness_df = pd.read_csv('world_happiness_report_2017.csv')

# Display the first few rows of the dataset
print(happiness_df.head())
```
### Extracting Python code
import pandas as pd

# Load the dataset
happiness_df = pd.read_csv('world_happiness_report_2017.csv')

# Display the first few rows of the dataset
print(happiness_df.head())
### Python function to run: import pandas as pd

# Load the dataset
happiness_df = pd.read_csv('world_happiness_report_2017.csv')

# Display the f